### Search for assets and observations in Collector for ArcGIS

This notebook demonstrates how you can configure search in the layer of a web maps. The search is then honored in apps, including Collector and Explorer.


#### Connect to your ArcGIS Online or ArcGIS Enterprise organization

In [ ]:
import arcgis
from arcgis.gis import GIS
from arcgis.mapping import WebMap
import json

print('Enter your ArcGIS account user name: ')
username = input()
gis = GIS('https://www.arcgis.com', username) # the item IDs used for maps here are available to user `kdonia_nitro`
print('Connected to {}'.format(gis.properties.portalHostname))

#### Get the web map

Search by ID for the web map where you want to configure layer search. 

In [ ]:
# An item with a search configured
webmap_item_with_search = gis.content.get('2b6140162d1c479f9f60f80ea8d5ca1f') 

# An item without search configured
webmap_item_no_search = gis.content.get('e5bd1251a090491793f7b1b1b443ec30') # previously used 0005cebc80cf44b7b91e20a46611775a

#### Search a feature layer

If the map already has feature search configured, add another search to it. If not, insert the first feature search.

In [ ]:
def add_layer_search(webmap_item, layer_url, field_name, exact_match, esri_field_type):
    json_data = webmap_item.get_data()
    if 'applicationProperties' not in json_data:
        print("Bad JSON -- OOPS!")
        return
    
    app_properties_viewing = json_data['applicationProperties']['viewing']

    # Get the ID of the layer to search, based on the URL you know it has
    webmap = WebMap(webmap_item)
    layers = webmap.layers
    layer_id = None
    for layer in layers:
        if layer.url == layer_url:
            layer_id = layer.id
        
    if layer_id is None:
        print('Layer URL not found: ' + layer_url, end='\n\n')
        return
         
    # if there isn't feature search defined, add the property with the defaults so that the next section can add the layer info
    if 'search' not in app_properties_viewing:
        search_property = {'enabled': True, 'disablePlaceFinder': False, 'hintText': 'Place or Address', 'layers': [{'id': layer_id, 'field': {'name': field_name, 'exactMatch': exact_match, 'type': esri_field_type}}]}
        app_properties_viewing['search'] = search_property
    
        print("Updated search for '" + webmap_item.title + "'")
        webmap_item.update(data = json.dumps(json_data))
            
    # else if there is already a feature search defined, add a layer to that search
    elif 'search' in app_properties_viewing:
 
        search_layers_property = app_properties_viewing['search']['layers']
        new_layer = {'id': layer_id, 'field': {'name': field_name, 'exactMatch': exact_match, 'type': esri_field_type}}
        search_layers_property.append(new_layer)
        webmap_item.update(data = json.dumps(json_data))
        print("Added another search in '" + webmap_item.title + "`", end='\n\n')
            

In [ ]:
webmap_item = webmap_item_no_search
layer_url = 'https://services.arcgis.com/N4jtru9dctSQR53c/arcgis/rest/services/Hydrant_maintenance/FeatureServer/0'
#layer_url = 'https://services.arcgis.com/N4jtru9dctSQR53c/arcgis/rest/services/Assets/FeatureServer/0'

add_layer_search(webmap_item, layer_url, 'Facility Identifier', True, 'esriFieldTypeInteger') #ID
add_layer_search(webmap_item, layer_url, 'Creator', False, 'esriFieldTypeString') #Details

#### Provide hint text

Check if it has layer search configured, and if so, provide hint text.

In [ ]:
webmap_item = webmap_item_with_search
json_data = webmap_item.get_data()
app_properties_viewing = json_data['applicationProperties']['viewing']

if ('search' in app_properties_viewing):
 
    layer_search_property = app_properties_viewing['search']
    hint_text = {'hintText': 'My search-specific hint text'}
    layer_search_property.update(hint_text)
    webmap_item.update(data = json.dumps(json_data))

    print("Updated hint text for '" + webmap_item.title + "'")   

#### Disable place search

If you won't want mobile workers searching for generic places and addresses, disable place search.

In [ ]:
webmap_item = webmap_item_with_search
json_data = webmap_item.get_data()
app_properties_viewing = json_data['applicationProperties']['viewing']

if ('search' in app_properties_viewing):
 
    layer_search_property = app_properties_viewing['search']
    disable_place_finder = {'disablePlaceFinder': True}
    layer_search_property.update(disable_place_finder)
    webmap_item.update(data = json.dumps(json_data))
    
    print("Disabled place finder for '" + webmap_item.title + "'")